## Import packages

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os

import cv2

from sklearn.model_selection import train_test_split

from keras.applications import Xception, VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

## Data Preparation

In [2]:
labels = ['Black-grass',
          'Charlock',
          'Cleavers',
          'Common Chickweed',
          'Common wheat',
          'Fat Hen',
          'Loose Silky-bent',
          'Maize',
          'Scentless Mayweed',
          'Shepherds Purse',
          'Small-flowered Cranesbill',
          'Sugar beet']
img_size = 256
train_dir = '/kaggle/input/plant-seedlings-classification/train/'
test_dir = '/kaggle/input/plant-seedlings-classification/test/'
sample_submission = pd.read_csv('/kaggle/input/plant-seedlings-classification/sample_submission.csv')

## Training Data

In [3]:
train = []
for label_id, label in enumerate(labels):
    for file in os.listdir(os.path.join(train_dir, label)):
        train.append(['{}/{}'.format(label, file), label_id, label])
train = pd.DataFrame(train, columns=['file', 'label_id', 'label'])
train.head()

,file,label_id,label
0,Black-grass/957b8523c.png,0,Black-grass
1,Black-grass/6e193f1bb.png,0,Black-grass
2,Black-grass/0d1a9985f.png,0,Black-grass
3,Black-grass/5c405ae2d.png,0,Black-grass
4,Black-grass/ac56bd408.png,0,Black-grass


In [4]:
train.shape

(4750, 3)

## Image Pre-processing (Masking/Segmentation/Sharpening)

In [5]:
def create_mask_for_plant(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    sensitivity = 35
    lower_hsv = np.array([60 - sensitivity, 100, 50])
    upper_hsv = np.array([60 + sensitivity, 255, 255])

    mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

def segment_plant(image):
    mask = create_mask_for_plant(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output

def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp

In [6]:
train_data = []

for i in range(len(train)):
    img = cv2.imread(os.path.join(train_dir, train['file'][i]))
    img = cv2.resize(img,(img_size,img_size))
    img = segment_plant(img)
    img = sharpen_image(img)
    img = cv2.resize(img[:,:,:3], (img_size, img_size))
    train_data.append(np.array(img))

train_data = np.array(train_data)

### Label Encoding

In [7]:
train_label = train['label_id']
train_label =  to_categorical(train_label, 12)

## Train/Test set split

In [8]:
x_train, x_val, y_train, y_val = train_test_split(train_data, train_label, test_size = 0.1, random_state=10)

In [9]:
x_train.shape

(4275, 256, 256, 3)

In [10]:
y_train.shape

(4275, 12)

## Build Xception Model

In [11]:
base_model = Xception(weights = "imagenet", include_top=False, input_shape = (img_size, img_size, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

#compling and show model
model.compile(loss = "categorical_crossentropy", optimizer = 'Adadelta', metrics=["accuracy"])
model.summary()

83689472/83683744 [==============================] - 2s 0us/step
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 127, 127, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 127, 127, 32) 0           block1_conv1_bn[0][0]            
______________________

### Image Data Generator

In [12]:
datagen = ImageDataGenerator(rotation_range=180,
                             width_shift_range=0.3,
                             height_shift_range=0.3,
                             zoom_range=0.3,
                             horizontal_flip=True,
                             vertical_flip=True)

datagen.fit(x_train)

## Fit Model

In [13]:
batch_size = 16

In [14]:
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
          validation_data = (x_val,y_val),
          steps_per_epoch = len(x_train)//batch_size,
          epochs = 100,
          verbose = 1)
model.save_weights("/kaggle/working/Xception.h5")

Epoch 1/100
267/267 [==============================] - 99s 369ms/step - loss: 2.5287 - accuracy: 0.0916 - val_loss: 2.4867 - val_accuracy: 0.1432
Epoch 2/100
267/267 [==============================] - 96s 360ms/step - loss: 2.4849 - accuracy: 0.1136 - val_loss: 2.6159 - val_accuracy: 0.1895
Epoch 3/100
267/267 [==============================] - 97s 362ms/step - loss: 2.4594 - accuracy: 0.1221 - val_loss: 2.7883 - val_accuracy: 0.2274
Epoch 4/100
267/267 [==============================] - 97s 362ms/step - loss: 2.4337 - accuracy: 0.1524 - val_loss: 2.7917 - val_accuracy: 0.2589
Epoch 5/100
267/267 [==============================] - 96s 358ms/step - loss: 2.4135 - accuracy: 0.1634 - val_loss: 2.8470 - val_accuracy: 0.2884
Epoch 6/100
267/267 [==============================] - 97s 361ms/step - loss: 2.3852 - accuracy: 0.1780 - val_loss: 2.8495 - val_accuracy: 0.3179
Epoch 7/100
267/267 [==============================] - 95s 357ms/step - loss: 2.3539 - accuracy: 0.2078 - val_loss: 2.9821 -

Epoch 57/100
267/267 [==============================] - 98s 365ms/step - loss: 0.8508 - accuracy: 0.7408 - val_loss: 0.5752 - val_accuracy: 0.8379
Epoch 58/100
267/267 [==============================] - 96s 358ms/step - loss: 0.8269 - accuracy: 0.7492 - val_loss: 0.5656 - val_accuracy: 0.8379
Epoch 59/100
267/267 [==============================] - 95s 354ms/step - loss: 0.8185 - accuracy: 0.7549 - val_loss: 0.5507 - val_accuracy: 0.8400
Epoch 60/100
267/267 [==============================] - 94s 351ms/step - loss: 0.8048 - accuracy: 0.7516 - val_loss: 0.5415 - val_accuracy: 0.8463
Epoch 61/100
267/267 [==============================] - 94s 351ms/step - loss: 0.8136 - accuracy: 0.7455 - val_loss: 0.5309 - val_accuracy: 0.8463
Epoch 62/100
267/267 [==============================] - 94s 350ms/step - loss: 0.7931 - accuracy: 0.7518 - val_loss: 0.5201 - val_accuracy: 0.8463
Epoch 63/100
267/267 [==============================] - 94s 352ms/step - loss: 0.7719 - accuracy: 0.7626 - val_loss: 0

## Prediction on Test Set

In [15]:
model.load_weights('/kaggle/working/Xception.h5')

In [22]:
test = []
for file in sample_submission['file']:
    test.append(os.path.join(test_dir,file))

test = pd.DataFrame(test, columns=['file'])
print(test.head(5))

                                                file
0  /kaggle/input/plant-seedlings-classification/t...
1  /kaggle/input/plant-seedlings-classification/t...
2  /kaggle/input/plant-seedlings-classification/t...
3  /kaggle/input/plant-seedlings-classification/t...
4  /kaggle/input/plant-seedlings-classification/t...


In [24]:
x_test = []

for i in range(len(test)):
    img = cv2.imread(test['file'][i])
    img = cv2.resize(img,(img_size,img_size))
    img_stack = segment_plant(img)
    img_stack = sharpen_image(img_stack)
    img = cv2.resize(img[:,:,:3], (img_size, img_size))
    x_test.append(np.array(img))
    
x_test = np.array(x_test)

In [26]:
Pred_labels = np.argmax(model.predict(x_test),axis = 1)
Pred_labels = pd.DataFrame(Pred_labels,index=None,columns=['species_num'])

In [29]:
sample_submission['species'] = [labels[i] for i in Pred_labels['species_num']]
sample_submission

,file,species
0,0021e90e4.png,Maize
1,003d61042.png,Maize
2,007b3da8b.png,Maize
3,0086a6340.png,Maize
4,00c47e980.png,Maize
...,...,...
789,fea355851.png,Maize
790,fea3da57c.png,Maize
791,fef2ade8c.png,Maize
792,ff65bc002.png,Maize


### Save File for Submission

In [30]:
sample_submission.to_csv('/kaggle/working/Xception-submission.csv',index=False)

## Test model on New Test Dataset (images downloaded from Google)

In [31]:
test_new_dir = '/kaggle/input/plant-seedlings-test-new/Test_new/'
Test_new = pd.read_csv('/kaggle/input/plant-seedlings-test-new/Test_new_actual.csv')

In [39]:
test_new = []
for file in Test_new['file']:
    test_new.append(os.path.join(test_new_dir,file))

test_new = pd.DataFrame(test_new, columns=['file'])
print(test_new.head(5))

                                                file
0  /kaggle/input/plant-seedlings-test-new/Test_ne...
1  /kaggle/input/plant-seedlings-test-new/Test_ne...
2  /kaggle/input/plant-seedlings-test-new/Test_ne...
3  /kaggle/input/plant-seedlings-test-new/Test_ne...
4  /kaggle/input/plant-seedlings-test-new/Test_ne...


In [40]:
x_test_new = []

for i in range(len(test_new)):
    img = cv2.imread(test_new['file'][i])
    img = cv2.resize(img,(img_size,img_size))
    img_stack = segment_plant(img)
    img_stack = sharpen_image(img_stack)
    img = cv2.resize(img[:,:,:3], (img_size, img_size))
    x_test_new.append(np.array(img))
    
x_test_new = np.array(x_test_new)

In [41]:
Pred_labels_new = np.argmax(model.predict(x_test_new),axis = 1)
Pred_labels_new = pd.DataFrame(Pred_labels_new,index=None,columns=['species_num'])

In [42]:
Test_new['prediction'] = [labels[i] for i in Pred_labels_new['species_num']]
Test_new

,file,species,prediction
0,3c75e7d79.png,Charlock,Charlock
1,456.jpg,Fat Hen,Scentless Mayweed
2,789.jpg,Charlock,Scentless Mayweed
3,123.jpg,Fat Hen,Fat Hen
4,131415.jpg,Small-flowered Cranesbill,Charlock
5,101112.jpg,Charlock,Cleavers
6,161718.jpg,Chickweed,Maize
7,da38229c7.png,Cleavers,Maize
8,202122.jpg,Cleavers,Scentless Mayweed
9,0ebf8f2f4.png,Maize,Maize


### The Xception Model get 5/20 correct

In [43]:
Test_new.to_csv('/kaggle/working/test_new_result_Xception.csv',index=False)